# **YoloV5(PyTorch) 모델 실습**

## **1. 사전 환경 세팅**

### **1-1. PyTorch 정상 설치 확인**

In [ ]:
%cd /workspace/yolov5

In [ ]:
!ls

import torch
print(torch.__version__)

### **1-2. Pre-trained weight로 추론 진행**

In [ ]:
# pretrained weight 인 yolov5l.pt 를 활용해 data/bus.jpg 이미지에 대해 추론 진행
# !python detect.py --weights yolov5s.pt --source /workspace/yolov5/data/images/bus.jpg
!python detect.py --weights yolov5s.pt --source /workspace/yolov5/data/images/zidane.jpg
                                            #    img.jpg                         # image
                                            #    vid.mp4                         # video
                                            #    screen                          # screenshot
                                            #    path/                           # directory
                                            #    list.txt                        # list of images
                                            #    list.streams                    # list of streams
                                            #    'path/*.jpg'                    # glob
                                            #    'https://youtu.be/LNwODJXcvt4'  # YouTube
                                            #    'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream

In [ ]:
# 추론 결과 시각화를 위한 모듈 로드
from IPython.display import Image, display
import glob
import os

# runs/detects/ 폴더 내 가장 최신 exp 확인
exp_dirs = sorted(glob.glob('runs/detect/exp*'), key=os.path.getmtime)
latest_exp_dir = exp_dirs[-1] if exp_dirs else None

# 폴더 내 추론된 이미지가 있을 경우 출력
if latest_exp_dir:
    result_images = glob.glob(os.path.join(latest_exp_dir, '*.jpg'))
    
    if result_images:
        display(Image(filename=result_images[0]))
    else:
        print("No result images found in the latest directory.")
else:
    print("No 'exp*' directories found.")

## **2. VOC 데이터셋 전처리**

In [ ]:
import os
from pathlib import Path
import xml.etree.ElementTree as ET
from tqdm import tqdm
import requests
import zipfile

# 함수: VOC 라벨을 YOLO 포맷으로 변환
def convert_label(annotation_path, label_path, class_names):
    def convert_box(size, box):
        dw, dh = 1. / size[0], 1. / size[1]
        x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
        return x * dw, y * dh, w * dw, h * dh

    tree = ET.parse(annotation_path)
    root = tree.getroot()
    size = root.find('size')
    width, height = int(size.find('width').text), int(size.find('height').text)

    with open(label_path, 'w') as out_file:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls in class_names and int(obj.find('difficult').text) != 1:
                xmlbox = obj.find('bndbox')
                box = [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')]
                bbox = convert_box((width, height), box)
                cls_id = class_names.index(cls)
                out_file.write(f"{cls_id} " + " ".join(map(str, bbox)) + '\n')

# 함수: URL에서 파일 다운로드
def download_file(url, dest_path):
    print(f"Downloading {url}...")
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(dest_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Saved to {dest_path}")

# 함수: VOC 데이터셋 다운로드 및 준비
def download_and_prepare_voc():
    base_dir = Path('./datasets/VOC')
    images_dir = base_dir / 'images'
    labels_dir = base_dir / 'labels'
    images_dir.mkdir(parents=True, exist_ok=True)
    labels_dir.mkdir(parents=True, exist_ok=True)

    # Pascal VOC 데이터셋 다운로드 URL
    voc_urls = {
        'VOCtrainval_06-Nov-2007': 'https://github.com/ultralytics/assets/releases/download/v0.0.0/VOCtrainval_06-Nov-2007.zip',
        'VOCtest_06-Nov-2007': 'https://github.com/ultralytics/assets/releases/download/v0.0.0/VOCtest_06-Nov-2007.zip',
        'VOCtrainval_11-May-2012': 'https://github.com/ultralytics/assets/releases/download/v0.0.0/VOCtrainval_11-May-2012.zip'
    }

    # 데이터셋 다운로드 및 압축 해제
    for name, url in voc_urls.items():
        zip_path = base_dir / f"{name}.zip"
        if not zip_path.exists():
            download_file(url, zip_path)

        # 압축 해제
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(images_dir)

    # VOCdevkit 디렉토리 확인
    voc_path = images_dir / 'VOCdevkit'
    if not voc_path.exists():
        raise FileNotFoundError("VOCdevkit directory not found after extraction.")

    # 클래스 목록
    class_names = [
        'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
        'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
        'train', 'tvmonitor'
    ]

    # 데이터셋 변환
    for year, image_set in [('2007', 'train'), ('2007', 'val'), ('2007', 'test'), ('2012', 'train'), ('2012', 'val')]:
        image_set_file = voc_path / f'VOC{year}/ImageSets/Main/{image_set}.txt'
        if not image_set_file.exists():
            print(f"Skipping {image_set} {year} - file not found: {image_set_file}")
            continue

        with open(image_set_file, 'r') as f:
            image_ids = f.read().strip().split()

        for image_id in tqdm(image_ids, desc=f"Processing {image_set} {year}"):
            annotation_path = voc_path / f'VOC{year}/Annotations/{image_id}.xml'
            image_path = voc_path / f'VOC{year}/JPEGImages/{image_id}.jpg'
            label_path = labels_dir / f"{image_set}_{year}_{image_id}.txt"

            if annotation_path.exists() and image_path.exists():
                # 이미지 복사 및 라벨 변환
                convert_label(annotation_path, label_path, class_names)
            else:
                print(f"Missing files for {image_id}: {annotation_path}, {image_path}")

# VOC 데이터셋 다운로드 및 준비 호출
# 현재 단계에서는 불필요
# download_and_prepare_voc()

## **3. Detector 정의**

In [ ]:
import os
import sys
from pathlib import Path
import torch
import yaml
import argparse
import numpy as np
from tqdm import tqdm

# YOLOv5 에 PYTHONPATH 지정
FILE = Path(__file__).resolve() if "__file__" in globals() else Path.cwd()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH

from models.yolo import Model
from utils.general import check_img_size, check_dataset, increment_path, colorstr
from utils.torch_utils import select_device
from utils.dataloaders import create_dataloader
from utils.loss import ComputeLoss

In [ ]:
# 모델 학습 파이프라인 정의
def train(hyp, opt, device):
    # 경로 설정
    save_dir = Path(opt.save_dir)  # 결과 저장 디렉토리
    epochs, batch_size, weights, data = opt.epochs, opt.batch_size, opt.weights, opt.data

    # 데이터셋 및 모델 설정 로드
    data_dict = check_dataset(data)  # data.yaml 파일 로드
    train_path, val_path = data_dict['train'], data_dict['val']  # 학습 및 검증 데이터 경로
    nc = int(data_dict['nc'])  # 클래스 수
    names = data_dict['names']  # 클래스 이름

    # 모델 생성
    model = Model(opt.cfg, ch=3, nc=nc).to(device)  # 새로운 모델 생성
    model.hyp = hyp  # 하이퍼파라미터 연결
    imgsz = check_img_size(opt.imgsz, s=32)  # 이미지 크기 확인

    # 데이터 로더 생성
    train_loader, dataset = create_dataloader(train_path,  # 학습 데이터 경로
                                              imgsz,  # 이미지 크기
                                              batch_size,  # 배치 크기
                                              32,  # 버퍼 크기
                                              hyp=hyp,  # 하이퍼파라미터
                                              augment=True,  # 데이터 증강 활성화
                                              cache=None,  # 캐싱 비활성화
                                              rect=False,  # 직사각형 모드 비활성화
                                              rank=-1,  # 분산 학습 비활성화
                                              workers=8,  # 워커 스레드 수
                                              image_weights=False,  # 이미지 가중치 비활성화
                                              prefix=colorstr('train: ')  # 로깅
                                              )

    # 옵티마이저 및 손실 함수 정의
    optimizer = torch.optim.SGD(model.parameters(),  # 모델 매개변수
                                 lr=hyp['lr0'],  # 초기 학습률
                                 momentum=hyp['momentum'],  # 모멘텀
                                 weight_decay=hyp['weight_decay'])  # 가중치 감소
    compute_loss = ComputeLoss(model)  # 손실 함수 초기화

    # 학습 루프
    for epoch in range(epochs):  # 에포크 반복
        model.train()  # 모델 학습 모드 설정
        for imgs, targets, paths, _ in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}'):  # 배치 반복
            imgs = imgs.to(device).float() / 255.0  # 이미지를 [0, 1]로 정규화
            targets = targets.to(device)  # 타겟을 디바이스로 전송

            # loss 계산
            pred = model(imgs)  # 예측값 계산
            loss, loss_items = compute_loss(pred, targets)  # 손실 계산

            # 역방향 계산 및 최적화
            optimizer.zero_grad()  # 그래디언트 초기화
            loss.backward()  # 역방향 전파
            optimizer.step()  # 매개변수 업데이트

        print(f'Epoch {epoch + 1}/{epochs} finished.')  # 에포크 완료 메시지 출력

    # 모델 저장
    torch.save(model.state_dict(), save_dir / 'last.pt')  # 모델 상태 저장

In [ ]:
# 하이퍼파라미터 정의
hyp = {
    'lr0': 0.01,  # 초기 학습률
    'momentum': 0.937,  # 모멘텀
    'weight_decay': 0.0005,  # 가중치 감소
    'box': 0.05,  # 박스 손실 가중치
    'cls': 0.5,  # 클래스 손실 가중치
    'obj': 1.0,  # 객체 손실 가중치
    'iou_t': 0.2,  # IoU 임계값
    'anchor_t': 4.0,  # 앵커 임계값
    'fl_gamma': 0.0,  # Focal Loss 감마값
    'hsv_h': 0.015,  # HSV 색조 증강 범위
    'hsv_s': 0.7,  # HSV 채도 증강 범위
    'hsv_v': 0.4,  # HSV 명도 증강 범위
    'degrees': 0.0,  # 회전 각도 증강 범위
    'translate': 0.1,  # 이동 증강 범위
    'scale': 0.5,  # 스케일 증강 범위
    'shear': 0.0,  # 전단 증강 범위
    'cls_pw': 1.0,  # 클래스 손실 가중치 파라미터
    'obj_pw': 1.0,  # 객체 손실 가중치 파라미터
    'mosaic': 1.0,  # 모자이크 증강 활성화 (기본값: 1.0)
    'copy_paste': 0.0,  # Copy-Paste 증강 비활성화 (기본값: 0.0)
    'perspective': 0.0,  # 원근법 증강 비활성화 (기본값: 0.0)
    'mixup': 0.0,  # MixUp 증강 비활성화 (기본값: 0.0)
    'flipud': 0.0,  # 세로 뒤집기 확률
    'fliplr': 0.5,  # 가로 뒤집기 확률
    # 필요한 경우 추가 증강 키를 여기에 추가...
}

# 옵션 정의
class Options:
    weights = ''  # 초기 가중치 경로
    cfg = '/workspace/yolov5/models/yolov5l.yaml'  # 모델 설정 파일 경로
    data = '/workspace/yolov5/data/VOC.yaml'  # 데이터셋 설정 파일 경로
    epochs = 100  # 학습 에포크 수
    batch_size = 8  # 모든 GPU의 총 배치 크기
    imgsz = 640  # 학습 및 검증 이미지 크기 (픽셀 단위)
    save_dir = 'runs/train'  # 결과 저장 디렉토리

opt = Options()

In [ ]:
if __name__ == '__main__':
    # GPU 사용 가능 시 GPU 선택, 그렇지 않으면 CPU 사용
    device = select_device('0' if torch.cuda.is_available() else 'cpu')
    
    # 고유한 저장 디렉토리 생성
    opt.save_dir = increment_path(Path(opt.save_dir) / 'exp')  # 저장 경로 증분
    opt.save_dir.mkdir(parents=True, exist_ok=True)  # 디렉토리 생성
    
    # 학습 함수 실행
    # train(hyp, opt, device)

## **4. 신규 학습 weight를 활용한 추론**

### **4-1. 사전 학습된 모델 로드 및 추론**

In [ ]:
# 4-1. 학습된 가중치 불러오기
# 이미 정의된 모델을 활용하여 가중치 로드
data = opt.data
data_dict = check_dataset(data)  # data.yaml 파일 로드
nc = int(data_dict['nc'])  # 클래스 수
model = Model(opt.cfg, ch=3, nc=nc).to(device)  # 새로운 모델 생성
model.hyp = hyp  # 하이퍼파라미터 연결
model.load_state_dict(torch.load(opt.save_dir / 'last.pt', map_location=device))  # 학습된 가중치 로드
model.eval()  # 평가 모드 설정
print("Trained weights loaded successfully!")

### **4-2. 이미지 로드 및 전처리**

In [ ]:
# 4-2. 이미지 로드 및 전처리
from PIL import Image
from torchvision.transforms import transforms
import cv2
import numpy as np

def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), stride=32, auto=True, scale_fill=False, scale_up=True):
    # 이미지 크기를 유지하면서 패딩 추가 (YOLO 입력 크기에 맞춤)
    shape = img.shape[:2]  # 원본 이미지의 높이, 너비
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # 비율 계산
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scale_up:  # 작은 이미지는 확대하지 않음
        r = min(r, 1.0)

    # 새 크기 계산
    ratio = r, r  # 너비, 높이 비율
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # 패딩 추가
    if auto:  # 32의 배수로 조정
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)
    elif scale_fill:  # 크기를 정확히 채움
        dw, dh = 0.0, 0.0
        new_unpad = new_shape
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]

    dw /= 2  # 패딩을 양쪽에 추가
    dh /= 2

    # 리사이즈 및 패딩
    if shape[::-1] != new_unpad:  # 새 크기로 리사이즈
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # 패딩 추가

    return img, ratio, (dw, dh)

# 추론할 이미지 경로
image_path = '/workspace/yolov5/data/images/bus.jpg'
img_size = opt.imgsz  # 옵션에서 설정된 입력 이미지 크기

# 이미지 로드 및 전처리
img_raw = cv2.imread(image_path)  # OpenCV로 이미지 로드 (PIL 대신)
img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
img_transformed, _, _ = letterbox(img_raw, new_shape=img_size, auto=False)  # letterbox 적용
img_tensor = transforms.ToTensor()(img_transformed).unsqueeze(0).to(device)  # 텐서로 변환 후 배치 차원 추가

print(f"Image preprocessed: {img_tensor.shape}")

### **4-3. 추론 함수 및 결과 시각화 함수 정의**

In [ ]:
# 4-3. 추론 함수 및 결과 시각화 함수 정의
import matplotlib.pyplot as plt
from PIL import ImageDraw

def run_inference(model, img_tensor):
    with torch.no_grad():  # 그래디언트 비활성화
        pred = model(img_tensor)  # 모델 추론
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)  # NMS 적용
    return pred

def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
    # Rescale coordinates (xyxy) from img1_shape to img0_shape
    if ratio_pad is None:  # Calculate from shapes
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])  # gain  = old / new
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2  # wh padding
    else:
        gain = ratio_pad[0]
        pad = ratio_pad[1]

    coords[:, [0, 2]] -= pad[0]  # x padding
    coords[:, [1, 3]] -= pad[1]  # y padding
    coords[:, :4] /= gain
    coords[:, :4] = coords[:, :4].clamp(min=0, max=img0_shape[1 if coords[:, 0].max() > 1 else 0])  # Clip bounding boxes
    return coords

def visualize_results(img_raw, pred, class_names, img_tensor_shape):
    # OpenCV에서 로드된 이미지를 PIL 이미지로 변환
    if isinstance(img_raw, np.ndarray):  # img_raw가 numpy.ndarray인지 확인
        img_raw = Image.fromarray(img_raw)

    draw = ImageDraw.Draw(img_raw)  # PIL 이미지 위에 그릴 도구 초기화
    for det in pred:  # 각 객체에 대해 처리
        if det is not None and len(det):
            det[:, :4] = scale_coords(img_tensor_shape[2:], det[:, :4], img_raw.size).round()  # 좌표 스케일 조정
            for *xyxy, conf, cls in det:
                label = f"{class_names[int(cls)]} {conf:.2f}"  # 클래스 및 신뢰도
                draw.rectangle(xyxy, outline="red", width=2)  # 경계 상자 그리기
                draw.text((xyxy[0], xyxy[1] - 10), label, fill="red")  # 클래스 이름 및 신뢰도 텍스트 추가
    return img_raw

### **4-4. 결과 시각화**

In [ ]:
# 4-4. 결과 시각화
class_names = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
    'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
    'train', 'tvmonitor'
]

predictions = run_inference(model, img_tensor)  # 추론 실행
result_img = visualize_results(img_raw, predictions, class_names, img_tensor.shape)  # 결과 시각화

# 이미지 출력
plt.figure(figsize=(10, 10))
plt.imshow(result_img)
plt.axis('off')
result_img.show() 